In [11]:
!python --version
!activate py37_32bit
import platform

print(f'bit_version : {platform.architecture()}')

Python 3.8.5
bit_version : ('64bit', 'WindowsPE')


In [12]:
# !python --version
!activate py37_32bit

In [ ]:
!python --version
!python 

Python 3.8.5


In [1]:
'''
1. 백테스트를 위한 데이터베이스 구축
- 데이터 조회
'''

# 패키지 호출 
import pandas as pd
import win32com.client
import sqlite3

In [6]:
# 대신증권 연결여부 체크

#objCpStatus = win32com.client.Dispatch('CpUtil.CpCybos')
objCpStatus = win32com.client.Dispatch("CpUtil.CpCybos")
print(objCpStatus)
# def check_connection():
#     bConnect = objCpStatus.IsConnect
#     if (bConnect == 0):
#         print("PLUS가 정상적으로 연결되지 않음.")
#         exit()
        
#     else:
#         print("연결완료")
        
#     return True

# check_connection()

com_error: (-2147221164, '클래스가 등록되지 않았습니다.', None, None)

In [5]:
# 주가 불러오기 - 갯수 기준
objStockChart = win32com.client.Dispatch("CpSysDib.StockChart")
 
objStockChart.SetInputValue(0, 'A005930')   #종목 코드 - 삼성전자

objStockChart.SetInputValue(1, ord('2')) # 개수로 조회
objStockChart.SetInputValue(4, 10) # 최근 100일 치

objStockChart.SetInputValue(5, [0, 2, 3, 4, 5, 8, 9]) #날짜,시가,고가,저가,종가,거래량
objStockChart.SetInputValue(6, ord('D')) # '차트 주가 - 일간 차트 요청
objStockChart.SetInputValue(9, ord('1')) # 수정주가 사용
objStockChart.BlockRequest()
 
len = objStockChart.GetHeaderValue(3)
 
print("날짜", "시가", "고가", "저가", "종가", "거래량")
print("==============================================")
 
for i in range(len):
    day = objStockChart.GetDataValue(0, i)
    open = objStockChart.GetDataValue(1, i)
    high = objStockChart.GetDataValue(2, i)
    low = objStockChart.GetDataValue(3, i)
    close = objStockChart.GetDataValue(4, i)
    vol = objStockChart.GetDataValue(5, i)
    print (day, open, high, low, close, vol)


날짜 시가 고가 저가 종가 거래량
20200422 48700 50000 48350 49850 18613864
20200421 49400 49700 48700 49250 27407543
20200420 51400 51400 50000 50100 21866354
20200417 50800 52000 50300 51400 32041675
20200416 49350 49350 48550 49000 23349760
20200414 48800 49200 48300 49000 14206216
20200413 48650 48900 48300 48300 14431800
20200410 48950 49250 48650 49250 17839111
20200409 49750 49800 48700 49100 22628058
20200408 49600 49750 48600 48600 25010314


In [9]:
# 주가 불러오기 - 날짜 기준
objStockChart = win32com.client.Dispatch("CpSysDib.StockChart")
 
objStockChart.SetInputValue(0, 'A005930') # 종목 코드 - 삼성전자
objStockChart.SetInputValue(1, ord('1')) # 날짜로 조회

objStockChart.SetInputValue(2, 0) # 종료 날짜, 0을 넣으면 가장 최근 날짜로 불러옴.
objStockChart.SetInputValue(3, 20200401) # 시작 날짜, 3월 1일로  설정하였음.

objStockChart.SetInputValue(5, [0, 2, 3, 4, 5, 8, 9]) # 날짜, 시가, 고가, 저가, 종가, 거래량, 거래대금
objStockChart.SetInputValue(6, ord('D')) # '차트 주가 - 일간 차트 요청
objStockChart.SetInputValue(9, ord('1')) # 수정주가 사용
objStockChart.BlockRequest()
 
len = objStockChart.GetHeaderValue(3)
 
print("날짜", "시가", "고가", "저가", "종가", "거래량", "거래대금")
print("==============================================")
 
for i in range(len):
    day = objStockChart.GetDataValue(0, i)
    open = objStockChart.GetDataValue(1, i)
    high = objStockChart.GetDataValue(2, i)
    low = objStockChart.GetDataValue(3, i)
    close = objStockChart.GetDataValue(4, i)
    vol = objStockChart.GetDataValue(5, i)
    vol_mon = objStockChart.GetDataValue(6, i)
    
    # 데이터 확인해보기
    print(day, open, high, low, close, vol, vol_mon)

날짜 시가 고가 저가 종가 거래량 거래대금
20200422 48700 50000 48350 49850 18613864 912223000000
20200421 49400 49700 48700 49250 27407543 1347415000000
20200420 51400 51400 50000 50100 21866354 1106552000000
20200417 50800 52000 50300 51400 32041675 1641068000000
20200416 49350 49350 48550 49000 23349760 1143760000000
20200414 48800 49200 48300 49000 14206216 694513000000
20200413 48650 48900 48300 48300 14431800 700511000000
20200410 48950 49250 48650 49250 17839111 872177000000
20200409 49750 49800 48700 49100 22628058 1111936000000
20200408 49600 49750 48600 48600 25010314 1230375000000
20200407 49650 50200 49000 49600 31524034 1561333000000
20200406 47500 48800 47250 48700 23395726 1123254000000
20200403 47400 47600 46550 47000 22784682 1074180000000
20200402 46200 46850 45350 46800 21621076 997831000000
20200401 47450 47900 45800 45800 27259532 1282483000000


In [11]:
'''
- DataFrame로 저장하기
'''

# 1. 주가 불러오기 - 날짜 기준
objStockChart = win32com.client.Dispatch("CpSysDib.StockChart")
 
objStockChart.SetInputValue(0, 'A005930') # 종목 코드 - 삼성전자
objStockChart.SetInputValue(1, ord('1')) # 날짜로 조회

objStockChart.SetInputValue(2, 0) # 종료 날짜, 0을 넣으면 가장 최근 날짜로 불러옴.
objStockChart.SetInputValue(3, 20200401) # 시작 날짜, 3월 1일로  설정하였음.

objStockChart.SetInputValue(5, [0, 2, 3, 4, 5, 8, 9]) # 날짜, 시가, 고가, 저가, 종가, 거래량, 거래대금
objStockChart.SetInputValue(6, ord('D')) # '차트 주가 - 일간 차트 요청
objStockChart.SetInputValue(9, ord('1')) # 수정주가 사용
objStockChart.BlockRequest()
 
len = objStockChart.GetHeaderValue(3)

# 2. 리스트 기반으로 저장
value_list = []

for i in range(len):
    day = objStockChart.GetDataValue(0, i)
    open = objStockChart.GetDataValue(1, i)
    high = objStockChart.GetDataValue(2, i)
    low = objStockChart.GetDataValue(3, i)
    close = objStockChart.GetDataValue(4, i)
    vol = objStockChart.GetDataValue(5, i)
    vol_mon = objStockChart.GetDataValue(6, i)
    
    # 데이터 확인해보기
    value_list.append([day, open, high, low, close, vol, vol_mon])

# 3, DataFrame로 변환
price_df = pd.DataFrame(value_list, columns = ['day','open', 'high', 'low', 'close', 'vol', 'vol_money'])

# 4. 데이터 확인
print(price_df)

         day   open   high    low  close       vol      vol_money
0   20200422  48700  50000  48350  49850  18613864   912223000000
1   20200421  49400  49700  48700  49250  27407543  1347415000000
2   20200420  51400  51400  50000  50100  21866354  1106552000000
3   20200417  50800  52000  50300  51400  32041675  1641068000000
4   20200416  49350  49350  48550  49000  23349760  1143760000000
5   20200414  48800  49200  48300  49000  14206216   694513000000
6   20200413  48650  48900  48300  48300  14431800   700511000000
7   20200410  48950  49250  48650  49250  17839111   872177000000
8   20200409  49750  49800  48700  49100  22628058  1111936000000
9   20200408  49600  49750  48600  48600  25010314  1230375000000
10  20200407  49650  50200  49000  49600  31524034  1561333000000
11  20200406  47500  48800  47250  48700  23395726  1123254000000
12  20200403  47400  47600  46550  47000  22784682  1074180000000
13  20200402  46200  46850  45350  46800  21621076   997831000000
14  202004

In [12]:
'''
방금 만든 데이터를 sqlite3 데이터베이스에 저장하기
'''

con = sqlite3.connect("price.db")
price_df.to_sql('A005930', con, if_exists='replace')